## Domácí úkol - Batoh

Za domácí úkol budete mít vyřešit pomocí evolučního algoritmu problém batohu. Ten spočívá v tom, že máme batoh kapacity K a N předmětů, každý s cenou c<sub>i</sub> a objemem v<sub>i</sub> a chceme vybrat takové věci, abychom maximalizovali zisk a zároveň abychom nepřekročili kapacitu batohu. 

Vstupní data máte ve složce *domaci_ukol_data*. Obsahuje čtyři soubory s daty a dva s výsledky. Na první řádce souboru s daty je vždy počet předmětů a kapacita batohu oddělené mezerou, každý další následující řádek obsahuje cenu a objem předmětu taktéž oddělené mezerou. První dva soubory slouží pro snažší odladění evolučního algoritmu a obsahují i k sobě extra soubory s optimálním řešením. Na dalších dvou máte za úkol algoritmus pustit a výsledky na nich naměřené mi poslat. 

Napište tedy nějaký svůj evoluční algoritmus, který bude řešit problém batohu a pusťte ho na vstupních datech. Svůj kód, popis evolučního algoritmu (zvolené evoluční operátory, kódování jedince, atd.) a rozbor výsledků, včetně nejlepšího dosaženého skóre i s jejich odůvodněním mi pošlete emailem do stanoveného deadline.  Pro sepsání popisu vašeho evolučního algoritmu, parametrů evoluce, zvolené reprezentace jedince a rozboru výsledků použijte [tento template](https://github.com/kackamac/Prirodou-inspirovane-algoritmy/blob/master/04_spojita_reprezentace/DU1_evolucni_algoritmy.pdf).

##### Importy

In [2]:
import array
import random
import numpy as np
import math

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import my_algorithms

Inicializácia random itemov a veľkosť batohu

In [3]:
# random.seed(64)
random.seed()
items_count = 20

weight_bounds = (10,200)
price_bounds = (10,200)

weights = random.sample(range(weight_bounds[0],weight_bounds[1]), items_count)
prices = random.sample(range(price_bounds[0],price_bounds[1]), items_count)

max_weight = 1000

Načítanie batohu zo súboru

In [4]:
#import weights and prices from txt file

def load_input_data(file_path):

    weights = []
    prices = []

    with open(file_path) as file:
        line_split = file.readline().split(" ")
        items_count = int(line_split[0])
        max_weight = int(line_split[1])
        for i in range(items_count):
            line_split = file.readline().split(" ")
            prices.append(int(line_split[0]))
            weights.append(int(line_split[1]))

    return weights, prices, max_weight

### Sedond try - permutation individual

max fitness - berieme itemy v poradí permutácie, kým nenaplníme batoh. Ostatné neberieme

In [5]:
def fitness_batoh_perm(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight > max_weight:
            return total_price,
        total_price += prices[i]
    return total_price,

In [6]:
def fitness_batoh_perm_2(individual):
    total_price = 0
    total_weight = 0
    for i in individual:
        total_weight += weights[i]
        if total_weight <= max_weight:
            total_price += prices[i]
        else:
            total_weight -= weights[i]
    return total_price,

In [7]:
def mut_transposition(individual, indv_size, transposition_count):
    count = random.randint(1, transposition_count)

    for i in range(count):
        pos_1 = random.randint(0, indv_size - 1)
        pos_2 = random.randint(0, indv_size - 1)

        individual[pos_1], individual[pos_2] = individual[pos_2], individual[pos_1]

    return individual,

In [15]:
weights, prices, max_weight = load_input_data(file_path = "./domaci_ukol_data/input_data_1000.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("indices", random.sample, range(indv_size), indv_size)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_batoh_perm)

toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", mut_transposition, indv_size=indv_size, transposition_count = 20)
#toolbox.register("select", tools.selRoulette)
toolbox.register("select", tools.selTournament, tournsize = 3)

In [16]:
POP_SIZE = 3_000
GEN_COUNT = 10_000


pop = toolbox.population(n=POP_SIZE)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.4, ngen=GEN_COUNT,
                               stats=stats, halloffame=hof, verbose=True)

print(hof)
print(fitness_batoh_perm(hof[0]))

gen	nevals	avg    	std    	min 	max  
0  	3000  	4685.69	1249.95	1302	10000
1  	2469  	5479.88	1179.49	2157	9776 
2  	2492  	5980.9 	1216.19	2089	9783 
3  	2438  	6386.8 	1297.72	1949	10466
4  	2467  	6692.19	1363.6 	1636	10743
5  	2515  	6937.67	1459.47	2032	11153
6  	2395  	7200.09	1488.2 	1594	11153
7  	2459  	7350.46	1560.68	2128	11651
8  	2427  	7546.68	1626.79	1409	11651
9  	2462  	7676.6 	1660.27	2204	11823
10 	2462  	7852.91	1736.92	2114	12719
11 	2449  	7978.04	1800.86	2028	13971
12 	2462  	8189.31	1855.33	1682	13971
13 	2500  	8295.47	1950.74	1905	13971
14 	2508  	8357.92	1964.36	2359	13971
15 	2469  	8474.55	1992.6 	1703	14323
16 	2461  	8608.63	2065.17	2469	14323
17 	2453  	8732.93	2128.95	1948	14323
18 	2424  	8856.56	2155.63	976 	14323
19 	2450  	8952.54	2223.56	1384	14323
20 	2483  	9123.67	2273.69	1564	14323
21 	2452  	9156.31	2339.34	1665	14323
22 	2449  	9336.06	2395.71	2180	14385
23 	2478  	9420.14	2422.38	2356	14867
24 	2403  	9516.71	2414.16	1096	15020
25 	2453  	9

KeyboardInterrupt: 

In [17]:
print(hof)
print(fitness_batoh_perm(hof[0]))

[[945, 32, 60, 446, 373, 599, 708, 48, 134, 476, 937, 830, 25, 657, 469, 610, 249, 494, 786, 421, 6, 53, 855, 836, 419, 732, 216, 137, 35, 743, 121, 849, 23, 603, 463, 480, 824, 146, 473, 347, 38, 37, 13, 986, 914, 989, 334, 382, 273, 703, 696, 281, 592, 493, 822, 770, 845, 987, 737, 426, 612, 886, 254, 379, 775, 215, 821, 967, 362, 573, 751, 736, 10, 887, 664, 161, 222, 453, 729, 863, 925, 604, 580, 935, 726, 457, 225, 54, 555, 286, 131, 112, 970, 243, 186, 701, 785, 435, 864, 789, 609, 944, 478, 73, 532, 106, 366, 694, 343, 554, 277, 75, 835, 993, 280, 119, 544, 692, 29, 725, 113, 143, 218, 810, 940, 99, 449, 781, 77, 531, 238, 269, 631, 913, 772, 208, 163, 139, 327, 69, 292, 357, 823, 78, 458, 846, 403, 670, 59, 901, 564, 90, 409, 997, 621, 686, 656, 148, 416, 129, 190, 315, 295, 165, 521, 947, 167, 920, 287, 74, 100, 897, 700, 181, 490, 516, 752, 812, 678, 811, 111, 92, 908, 943, 644, 477, 320, 921, 616, 88, 682, 484, 999, 638, 571, 551, 356, 205, 470, 832, 83, 965, 819, 309, 192, 

#### first try - binárny jedinci

max fitness (cena ju zvyšuje a váha prekročená cez max nosnosť batohu prináša postihy)

In [ ]:
def fitness_batoh_bin(individual):
    total_price = np.sum(np.array(individual) @ np.array(prices))
    total_weight = np.sum(np.array(individual) @ np.array(weights))

    if total_weight <= max_weight:
        return total_price,
    else:
        return 1,

Vytvorenie prostredia (knižnica Deap)

In [ ]:
weights, prices = load_input_data(file_path = "./domaci_ukol_data/input_data_100.txt")
indv_size = len(weights)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='b', fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, indv_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness_batoh_bin)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.2)
toolbox.register("select", tools.selRoulette)

Final part

In [ ]:
pop = toolbox.population(n=500)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=200, 
                               stats=stats, halloffame=hof, verbose=True)